<a href="https://colab.research.google.com/github/RafalDoroz/ai/blob/main/miary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_auc_score, matthews_corrcoef, classification_report
import numpy as np

# 1. Prawdziwe etykiety (y_true) i przewidywane etykiety (y_pred)
y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 0]  # Rzeczywiste klasy
y_pred = [1, 0, 1, 0, 0, 1, 0, 1, 1, 0]  # Przewidywane klasy

# 2. Macierz konfuzji
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()  # Rozpakowanie macierzy konfuzji

print("=== Macierz Konfuzji ===")
print(f"True Negative (TN): {tn} - Poprawne przewidywanie klasy negatywnej")
print(f"False Positive (FP): {fp} - Fałszywe przewidywanie klasy pozytywnej")
print(f"False Negative (FN): {fn} - Fałszywe przewidywanie klasy negatywnej")
print(f"True Positive (TP): {tp} - Poprawne przewidywanie klasy pozytywnej")

# 3. Wyjaśnienie miar
# (1) Dokładność
accuracy = accuracy_score(y_true, y_pred)
print("\n=== Dokładność (Accuracy) ===")
print(f"Dokładność: {accuracy:.2f}")
print("Definicja: Procent poprawnych przewidywań.")
print(f"Wzór: (TP + TN) / (TP + TN + FP + FN) = ({tp} + {tn}) / {len(y_true)}")

# (2) Precyzja
precision = precision_score(y_true, y_pred, zero_division=0)
print("\n=== Precyzja (Precision) ===")
print(f"Precyzja: {precision:.2f}")
print("Definicja: Procent przewidywanych pozytywnych, które są rzeczywiście pozytywne.")
print(f"Wzór: TP / (TP + FP) = {tp} / ({tp} + {fp})")

# (3) Czułość
recall = recall_score(y_true, y_pred, zero_division=0)
print("\n=== Czułość (Recall) ===")
print(f"Czułość: {recall:.2f}")
print("Definicja: Procent rzeczywistych pozytywnych, które zostały poprawnie wykryte.")
print(f"Wzór: TP / (TP + FN) = {tp} / ({tp} + {fn})")

# (4) F1-Score
f1 = f1_score(y_true, y_pred, zero_division=0)
print("\n=== F1-Score ===")
print(f"F1-Score: {f1:.2f}")
print("Definicja: Średnia harmoniczna precyzji i czułości.")
print(f"Wzór: 2 * (Precision * Recall) / (Precision + Recall)")

# (5) ROC AUC
roc_auc = roc_auc_score(y_true, y_pred)
print("\n=== ROC AUC ===")
print(f"ROC AUC: {roc_auc:.2f}")
print("Definicja: Miara zdolności modelu do rozróżniania klas (im bliżej 1, tym lepiej).")

# (6) Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_true, y_pred)
print("\n=== Matthews Correlation Coefficient (MCC) ===")
print(f"MCC: {mcc:.2f}")
print("Definicja: Miara korelacji pomiędzy przewidywaniami a rzeczywistymi klasami (od -1 do 1).")

# 4. Pełny raport klasyfikacji
print("\n=== Raport Klasyfikacji ===")
print(classification_report(y_true, y_pred, zero_division=0))


=== Macierz Konfuzji ===
True Negative (TN): 4 - Poprawne przewidywanie klasy negatywnej
False Positive (FP): 1 - Fałszywe przewidywanie klasy pozytywnej
False Negative (FN): 1 - Fałszywe przewidywanie klasy negatywnej
True Positive (TP): 4 - Poprawne przewidywanie klasy pozytywnej

=== Dokładność (Accuracy) ===
Dokładność: 0.80
Definicja: Procent poprawnych przewidywań.
Wzór: (TP + TN) / (TP + TN + FP + FN) = (4 + 4) / 10

=== Precyzja (Precision) ===
Precyzja: 0.80
Definicja: Procent przewidywanych pozytywnych, które są rzeczywiście pozytywne.
Wzór: TP / (TP + FP) = 4 / (4 + 1)

=== Czułość (Recall) ===
Czułość: 0.80
Definicja: Procent rzeczywistych pozytywnych, które zostały poprawnie wykryte.
Wzór: TP / (TP + FN) = 4 / (4 + 1)

=== F1-Score ===
F1-Score: 0.80
Definicja: Średnia harmoniczna precyzji i czułości.
Wzór: 2 * (Precision * Recall) / (Precision + Recall)

=== ROC AUC ===
ROC AUC: 0.80
Definicja: Miara zdolności modelu do rozróżniania klas (im bliżej 1, tym lepiej).

=== Ma